<a href="https://colab.research.google.com/github/roitraining/SparkProgram/blob/Day3/Day3/Ch04_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Open a terminal window and run the following commands:
sudo bash
start-hadoop
cd /home/student/ROI/SparkProgram/Day3
./fixhive.sh


## Let's make a simple hive table for regions.

In [ ]:
! cat /class/regions.hql
! /class/fixhive.sh


In [1]:
import sys
sys.path.append('/class')
from initspark import *
sc, spark, conf = initspark()


initializing pyspark
pyspark initialized


## You can query an existing Hive table and bring it into a Spark DataFrame.

In [2]:
regions = spark.sql('select * from regions')
regions.show()
print(regions)


+--------+----------+
|regionid|regionname|
+--------+----------+
|       1|   Eastern|
|       2|   Western|
|       3|  Northern|
|       4|  Southern|
+--------+----------+

DataFrame[regionid: int, regionname: string]


## Read in a file to a Spark DataFrame.

In [3]:
territories = spark.read.csv('/class/datasets/northwind/CSVHeaders/territories', header=True)
territories.show()


+-----------+-------------+--------+
|TerritoryID|TerritoryName|RegionID|
+-----------+-------------+--------+
|      01581|     Westboro|       1|
|      01730|      Bedford|       1|
|      01833|    Georgetow|       1|
|      02116|       Boston|       1|
|      02139|    Cambridge|       1|
|      02184|    Braintree|       1|
|      02903|   Providence|       1|
|      03049|       Hollis|       3|
|      03801|   Portsmouth|       3|
|      06897|       Wilton|       1|
|      07960|   Morristown|       1|
|      08837|       Edison|       1|
|      10019|     New York|       1|
|      10038|     New York|       1|
|      11747|     Mellvile|       1|
|      14450|     Fairport|       1|
|      19428| Philadelphia|       3|
|      19713|       Neward|       1|
|      20852|    Rockville|       1|
|      27403|   Greensboro|       1|
+-----------+-------------+--------+
only showing top 20 rows



## Use createOrReplaceTempView to create a virtual table in the Hive catalog and then it can be queried using SQL as if it were a hive table.

In [4]:
territories.createOrReplaceTempView('territories')
t1 =spark.sql('select * from territories where regionid = 1')
t1.show()
print(t1.count())


+-----------+-------------+--------+
|TerritoryID|TerritoryName|RegionID|
+-----------+-------------+--------+
|      01581|     Westboro|       1|
|      01730|      Bedford|       1|
|      01833|    Georgetow|       1|
|      02116|       Boston|       1|
|      02139|    Cambridge|       1|
|      02184|    Braintree|       1|
|      02903|   Providence|       1|
|      06897|       Wilton|       1|
|      07960|   Morristown|       1|
|      08837|       Edison|       1|
|      10019|     New York|       1|
|      10038|     New York|       1|
|      11747|     Mellvile|       1|
|      14450|     Fairport|       1|
|      19713|       Neward|       1|
|      20852|    Rockville|       1|
|      27403|   Greensboro|       1|
|      27511|         Cary|       1|
|      40222|   Louisville|       1|
+-----------+-------------+--------+

19


## Spark DataFrames can be saved to a Hive table using either the saveAsTable method or writing a SQL query that uses CREATE TABLE AS.

In [10]:
territories.write.saveAsTable('Territories2', mode='overwrite')
spark.sql('create table Territories3 as select * from territories')


DataFrame[]

### Confirm that the tables exist in Hive
From the command line type hive or spark-sql and then show tables;


In [ ]:
#! hive -e "show tables;"
! spark-sql "show tables;"

## Queries use standard HQL to mix Hive tables and virtual tables. Both are read into a Spark DataFrame and the processing happens at the Spark level, not at the Hive level. HQL is just used to parse the logic into the corresponding Spark methods.

In [12]:
sql = """
select r.regionid, r.regionname, t.territoryid, t.territoryname 
from regions as r 
join territories as t on r.regionid = t.regionid 
order by r.regionid, t.territoryid
"""
rt = spark.sql(sql)
rt.show(10)

tr = regions.join(territories, regions.regionid == territories.RegionID). \
     select('regions.regionid', 'regionname', 'TerritoryID', 'TerritoryName') .\
    orderBy('regionid', 'territoryid')
tr.show(10)


+--------+----------+-----------+-------------+
|regionid|regionname|territoryid|territoryname|
+--------+----------+-----------+-------------+
|       1|   Eastern|      01581|     Westboro|
|       1|   Eastern|      01730|      Bedford|
|       1|   Eastern|      01833|    Georgetow|
|       1|   Eastern|      02116|       Boston|
|       1|   Eastern|      02139|    Cambridge|
|       1|   Eastern|      02184|    Braintree|
|       1|   Eastern|      02903|   Providence|
|       1|   Eastern|      06897|       Wilton|
|       1|   Eastern|      07960|   Morristown|
|       1|   Eastern|      08837|       Edison|
+--------+----------+-----------+-------------+
only showing top 10 rows

+--------+----------+-----------+-------------+
|regionid|regionname|TerritoryID|TerritoryName|
+--------+----------+-----------+-------------+
|       1|   Eastern|      01581|     Westboro|
|       1|   Eastern|      01730|      Bedford|
|       1|   Eastern|      01833|    Georgetow|
|       1|   E

## LAB: ## 
### Read the northwind JSON products and make it into a TempView and do the same with the CSVHeaders version of categories.
<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Look at Day 2 or below to copy the code to load in the DataFrames
<br>
Turn each DataFrame into a temporary view
<br>
<br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
categories = spark.read.load('/class/datasets/northwind/CSV/categories', format = 'csv', sep = ',', inferSchema = True, header = False)

prodSchema = StructType([
    StructField('productid', IntegerType()), 
    StructField('productname', StringType()),
    StructField('supplierid', IntegerType()), 
    StructField('categoryid', IntegerType()), 
    StructField('quantityperunit', StringType()), 
    StructField('unitprice', FloatType()), 
    StructField('unitsinstock', IntegerType()), 
    StructField('unitsonorder', IntegerType()), 
    StructField('reorderlevel', IntegerType()), 
    StructField('discontinued', IntegerType())
])
products = spark.read.json('/class/datasets/northwind/JSON/products', schema=prodSchema)

categories.createOrReplaceTempView('categories')
products.createOrReplaceTempView('products')```
</p>
</details>

## Install the MySQL Python connector. This has nothing to do with Spark, but if you want to run SQL queries directly it is helpful.
It's already on our machines so we don't need to run this now

In [ ]:
! pip install mysql-connector-python

## Let's make sure we have a database for northwind and no regions table.

In [14]:
import mysql.connector
try:
    cn = mysql.connector.connect(host='localhost', user='test', password='password')
    cursor = cn.cursor()
    cursor.execute('create database if not exists northwind')
    cn.close()

    cn = mysql.connector.connect(host='localhost', user='test', password='password', database='northwind')
    cursor = cn.cursor()    
    cursor.execute('drop table if exists regions')
    cn.close()
except:
    print('something went wrong')
else:
    print('success')



success


## Write a DataFrame to a SQL database.

In [15]:
regions.write.format("jdbc").options(url="jdbc:mysql://localhost/northwind", \
                                     driver='com.mysql.jdbc.Driver', \
                                     dbtable='regions', \
                                     user='test', password = "password", mode = "append", \
                                     useSSL = "false").save()


## Read a SQL table into a Spark DataFrame.

In [38]:
regions2 = spark.read.format("jdbc").options(url="jdbc:mysql://localhost/northwind", \
                                             driver="com.mysql.jdbc.Driver", \
                                             dbtable= "regions", \
                                             user="test", password="password").load()
regions2.show()


+--------+----------+
|regionid|regionname|
+--------+----------+
|       1|   Eastern|
|       2|   Western|
|       3|  Northern|
|       4|  Southern|
+--------+----------+



### If you don't wont to bring an entire SQL table into a spark DataFrame use query instead of dbtable

In [37]:
regions3 = spark.read.format("jdbc").options(url="jdbc:mysql://localhost/northwind", \
                                             driver="com.mysql.jdbc.Driver", \
                                             query= "select * from regions where regionid < 3", \
                                             user="test", password="password").load()
regions3.show()



+--------+----------+
|regionid|regionname|
+--------+----------+
|       1|   Eastern|
|       2|   Western|
+--------+----------+



## Creating the regions2 DataFrame does not execute anything yet, but by making the DataFrame into a Temp View then running a Spark SQL query, it tells Spark to read the SQL data into a DataFrame and then use the cluster to do the processing, not the SQL source.

In [17]:
regions2.createOrReplaceTempView('regions2')
spark.sql('select * from regions2 where regionid < 3').show()

+--------+----------+
|regionid|regionname|
+--------+----------+
|       1|   Eastern|
|       2|   Western|
+--------+----------+



In [18]:
spark.read.table('regions2').where('regionid < 3').show()

+--------+----------+
|regionid|regionname|
+--------+----------+
|       1|   Eastern|
|       2|   Western|
+--------+----------+



## Alternate ways to code a query using SQL and methods.

In [19]:
print(spark.sql('select count(*) from regions').collect())
spark.sql('select * from regions').count()

[Row(count(1)=4)]


4

## Using SQL you can use familiar syntax instead of withColumn or withCoumnRenamed methods.
Note the expr function needs to be imported when you want to use a stringified SQL function using dot syntax

In [31]:
from pyspark.sql.functions import expr

t1 = spark.sql('select TerritoryID as TerrID, UPPER(TerritoryName) as TerritoryName, RegionID from territories')
t1.show(5)

from pyspark.sql.functions import expr
territories.withColumn('TerritoryName', expr('UPPER(TerritoryName)')).withColumnRenamed('TerritoryID', 'TerrID').show(5)


+------+-------------+--------+
|TerrID|TerritoryName|RegionID|
+------+-------------+--------+
| 01581|     WESTBORO|       1|
| 01730|      BEDFORD|       1|
| 01833|    GEORGETOW|       1|
| 02116|       BOSTON|       1|
| 02139|    CAMBRIDGE|       1|
+------+-------------+--------+
only showing top 5 rows

+------+-------------+--------+
|TerrID|TerritoryName|RegionID|
+------+-------------+--------+
| 01581|     WESTBORO|       1|
| 01730|      BEDFORD|       1|
| 01833|    GEORGETOW|       1|
| 02116|       BOSTON|       1|
| 02139|    CAMBRIDGE|       1|
+------+-------------+--------+
only showing top 5 rows



In [34]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

# This won't work though if you want to use python functions, you need to go another step
# territories.withColumn('TerritoryName', territories.TerritoryName.upper()).show()

# You need to make the python function callable by spark by wrapping it in the udf function
# which tells spark what datatype it returns
territories.withColumn('TerritoryName', udf(str.upper, StringType())(territories.TerritoryName)).show()



+-----------+-------------+--------+
|TerritoryID|TerritoryName|RegionID|
+-----------+-------------+--------+
|      01581|     WESTBORO|       1|
|      01730|      BEDFORD|       1|
|      01833|    GEORGETOW|       1|
|      02116|       BOSTON|       1|
|      02139|    CAMBRIDGE|       1|
|      02184|    BRAINTREE|       1|
|      02903|   PROVIDENCE|       1|
|      03049|       HOLLIS|       3|
|      03801|   PORTSMOUTH|       3|
|      06897|       WILTON|       1|
|      07960|   MORRISTOWN|       1|
|      08837|       EDISON|       1|
|      10019|     NEW YORK|       1|
|      10038|     NEW YORK|       1|
|      11747|     MELLVILE|       1|
|      14450|     FAIRPORT|       1|
|      19428| PHILADELPHIA|       3|
|      19713|       NEWARD|       1|
|      20852|    ROCKVILLE|       1|
|      27403|   GREENSBORO|       1|
+-----------+-------------+--------+
only showing top 20 rows



## If you want to use a function that is not a standard python or SQL function, you can always create one in python and make it callable from spark

In [33]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def inventoryvalue(quantity, price):
    return quantity * price

# Turn the Python function into a Spark callable function
invvalue = udf(inventoryvalue, FloatType())
p = products
p2 = p.withColumn('value', invvalue(p.unitsinstock, p.unitprice))
display(p2)




,productid,productname,supplierid,categoryid,quantityperunit,unitprice,unitsinstock,unitsonorder,reorderlevel,discontinued,value
0,1,Chai,8,1,10 boxes x 30 bags,18.00,39,0,10,1,702.0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,1,323.0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0,130.0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0,1166.0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1,0.0
5,6,Grandma's Boysenberry Spread,3,2,12 - 8 oz jars,25.00,120,0,25,0,3000.0
6,7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30.00,15,0,10,0,450.0
7,8,Northwoods Cranberry Sauce,3,2,12 - 12 oz jars,40.00,6,0,0,0,240.0
8,9,Mishi Kobe Niku,4,6,18 - 500 g pkgs.,97.00,29,0,0,1,2813.0
9,10,Ikura,4,8,12 - 200 ml jars,31.00,31,0,0,0,961.0


### Python decorators are an even better option

In [ ]:
@udf(FloatType())
def inventoryvalue(quantity, price):
    return quantity * price

p2 = p.withColumn('value', inventoryvalue(p.unitsinstock, p.unitprice))
display(p2)


In [35]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def inventoryvalue(quantity, price):
    return quantity * price

# Or dynamically wrap it, but it's harder to read
p2 = p.withColumn('value', udf(inventoryvalue, FloatType())(p.unitsinstock, p.unitprice))
display(p2)



,productid,productname,supplierid,categoryid,quantityperunit,unitprice,unitsinstock,unitsonorder,reorderlevel,discontinued,value
0,1,Chai,8,1,10 boxes x 30 bags,18.00,39,0,10,1,702.0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,1,323.0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0,130.0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0,1166.0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1,0.0
5,6,Grandma's Boysenberry Spread,3,2,12 - 8 oz jars,25.00,120,0,25,0,3000.0
6,7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30.00,15,0,10,0,450.0
7,8,Northwoods Cranberry Sauce,3,2,12 - 12 oz jars,40.00,6,0,0,0,240.0
8,9,Mishi Kobe Niku,4,6,18 - 500 g pkgs.,97.00,29,0,0,1,2813.0
9,10,Ikura,4,8,12 - 200 ml jars,31.00,31,0,0,0,961.0


## To make it easier though, you could make the Python function into a udf that SQL can understand similar to how you can make a DataFrame seem like a virtual table with createOrReplaceTempView.

In [36]:
def reverseString(x):
    return x[::-1]

spark.udf.register('reverse', reverseString, StringType())

spark.sql('select *, reverse(TerritoryName) as Reversed from Territories').orderBy('Reversed').show()

+-----------+-------------+--------+------------+
|TerritoryID|TerritoryName|RegionID|    Reversed|
+-----------+-------------+--------+------------+
|      90405| Santa Monica|       2|acinoM atnaS|
|      29202|     Columbia|       4|    aibmuloC|
|      19428| Philadelphia|       3|aihpledalihP|
|      33607|        Tampa|       4|       apmaT|
|      95054|  Santa Clara|       2| aralC atnaS|
|      30346|      Atlanta|       4|     atnaltA|
|      48075|   Southfield|       3|  dleifhtuoS|
|      98052|      Redmond|       2|     dnomdeR|
|      44122|    Beachwood|       3|   doowhcaeB|
|      19713|       Neward|       1|      draweN|
|      01730|      Bedford|       1|     drofdeB|
|      02903|   Providence|       1|  ecnedivorP|
|      02184|    Braintree|       1|   eertniarB|
|      02139|    Cambridge|       1|   egdirbmaC|
|      85251|   Scottsdale|       2|  eladsttocS|
|      11747|     Mellvile|       1|    elivlleM|
|      55113|    Roseville|       3|   ellivesoR|


## HQL has collect_set and collect_list functions to aggregate items into a list instead of summing them up. 

In [ ]:
from pyspark.sql.functions import collect_list
territories.groupBy(territories.RegionID).agg(collect_list(territories.TerritoryName)).show()

tr1 = spark.sql("SELECT RegionID, collect_list(TerritoryName) AS TerritoryList FROM Territories GROUP BY RegionID")
tr1.show()
tr1.printSchema()
print(tr1.take(1))


## Instead of a simple datatype, you could also collect complex structured objects using the HQL NAMED_STRUCT.

In [ ]:
sql = """
SELECT r.RegionID, r.RegionName
, COLLECT_SET(NAMED_STRUCT("TerritoryID", TerritoryID, "TerritoryName", TerritoryName)) AS TerritoryList
FROM Regions AS r
JOIN Territories AS t ON r.RegionID = t.RegionID
GROUP BY r.RegionID, r.RegionName
ORDER BY r.RegionID
"""

tr2 = spark.sql(sql)
tr2.printSchema()
print(tr2)
tr2.show()
print(tr2.take(2))
tr2.write.json('TerritoryRegion.json')
spark.sql('create table TerritoryRegion as ' + sql)

## If you have data that is already collected into a complex datatype and want to flatten it, you could use HQL EXPLODE function.

## You could use the Spark explode method.

In [ ]:
from pyspark.sql.functions import explode
tr1.select('RegionID', explode('TerritoryList')).show()


## Or if the DataFrame is turned into a Temp View, you could use the HQL query to do it.

In [ ]:
tr1.createOrReplaceTempView('RegionTerritories')
sql = """
SELECT RegionID, TerritoryName
FROM RegionTerritories
LATERAL VIEW EXPLODE(TerritoryList) EXPLODED_TABLE AS TerritoryName
ORDER BY RegionID, TerritoryName
"""
spark.sql(sql).show()

## Or you could select specific elements from a collection.

In [ ]:
tr2.createOrReplaceTempView('RegionTerritories')
spark.sql("select RegionId, RegionName, TerritoryList[0] as First, TerritoryList[size(TerritoryList) - 1] as Last, size(TerritoryList) as TerritoryCount from RegionTerritories").show()


## If the array is of structs, note the syntax of fetching the elements from the struct uses the . like an object property.

In [ ]:
sql = """
SELECT RegionID, RegionName, Territory.TerritoryID AS TerritoryID
, Territory.TerritoryName AS TerritoryName
FROM RegionTerritories
LATERAL VIEW EXPLODE(TerritoryList) EXPLODED_TABLE AS Territory
"""
spark.sql(sql).show()


## HOMEWORK: ## 
**First Challenge**

Create a Python function to determine if a number is odd or even and use that to select only the even numbered shippers from the TSV folder of northwind. Note the TSV file does not have headers so you will need to do something to make the DataFrame have a meaningful structure. I would suggest using Spark SQL as much as possible to rename and cast the columns which are ShipperID, CompanyName, and Phone.

**Second Challenge**

Take the Order_LineItems.json folder, read it into a DataFrame, and flatten it and then calculate the average price paid for a product.

<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Take a look at the MakeOrders_LineItems.py file provided to see how the Order_LineItems.json was generated in the first place
<br>
Use modulus with remainder of zero to determine if something is even
<br>
Use udf to make a version of the function that is callable using dot syntax and udf.register to make a version callable from within a SQL string
<br>
Use LATERAL VIEW EXPLODE() EXPLODED_TABLE to flatten out the nested format file
<br>
Once flattened do a traditional aggregate to calculate the average.
<br>
<br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
```
</p>
</details>




# End of notebook